# Higgs Boson Discovery

2023-03-14. Because of this date, I include the following sub-header.

## rediscovery (?), re-analysis (?), one more time (?)

A nice visualization to take me back to grad school - where I worked at [BNL](https://www.bnl.gov) / [RHIC](https://www.bnl.gov/rhic/) / [PHENIX](https://www.phenix.bnl.gov/) ([archived](https://web.archive.org/web/20230314100142/https://www.bnl.gov/world/) / [archived](https://web.archive.org/web/20230312202521/https://www.bnl.gov/rhic/) / [archived](https://web.archive.org/web/20230314191656/https://www.phenix.bnl.gov/)) which basically means I worked at a government particle accelerator. I wasn't at [CERN](https://home.cern/) / [LHC]() / [CMS](https://home.cern/science/experiments/cms) ([archived](https://web.archive.org/web/20230314192144/https://home.cern/) / [archived]() / [archived](https://web.archive.org/web/20230314192110/https://home.cern/science/experiments/cms)), which is one of the experiments contributing to the discovery of the Higgs boson and the provider of the data I'll be using. I did participate in _some_ collaboration with LHC groups. I was even invited to the [4<super>th</super> of July Discovery Announcement](https://en.wikipedia.org/wiki/Higgs_boson#Search_before_4_July_2012) ([archived](https://web.archive.org/web/20230314192829/https://en.wikipedia.org/wiki/Higgs_boson)), but I got my keys locked in my car while hanging out with friends halfway to Columbia University. (I think reality makes for a better story - we got in my car using tree branches, but it was long past the midnight announcement time.)

The results (if I accomplish my goal)should look something like what follows. The image comes from https://arxiv.org/format/1207.7235, the discovery paper for the Higgs.

<br/>
<div>
  <img src="./publication_4lepton_spectrum.png"
       alt="Publication histogram - our goal for the analysis is to be similar to this"
       width="400px">
</div>
<br/>

## Setup

Using `conda` for environment setup. This time, because of computer availability, I did it on the Windows CMD prompt (`Anaconda Prompt (miniconda3)`, specifically). `Jupyter` makes for simpler visualizations, so here I am.

### When are we? (good for experimental notebooks)

```
>powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]{5}_', '_'
1678797612_20230314T124012-0600

>::  Or, in case you don't have access to PowerShell,
>::+ you can extract that from the following mess.
>::
>(set LOCALE_INFO=) && (echo( > nul) && (for /f "usebackq tokens=*" %k in (`systeminfo ^| findstr ";" ^| cmd /q /v:on /c "set/p .=&echo(!.!"`) do @if not defined LOCALE_INFO set "LOCALE_INFO=%k") && (echo() && (for /f "tokens=*" %i in ('tzutil /g') do @echo "%date%" "%time%" "%i" "%LOCALE_INFO%") && (echo() && (w32tm /tz) && (echo()

"Tue 03/14/2023" "12:39:37.15" "Mountain Standard Time" "System Locale:             en-us;English (United States)"

Time zone: Current:TIME_ZONE_ID_DAYLIGHT Bias: 420min (UTC=LocalTime+Bias)
  [Standard Name:"Mountain Standard Time" Bias:0min Date:(M:11 D:1 DoW:0)]
  [Daylight Name:"Mountain Daylight Time" Bias:-60min Date:(M:3 D:2 DoW:0)]


>::  Gosh, that makes me miss bash
```
### Environment stuff, `pip` installs, and on to Jupyter 

```
>conda create --name higgs_boson python=3.10
>conda activate higgs_boson
(higgs_boson)>
(higgs_boson)>python -m pip install --upgrade pip
(higgs_boson)>pip install jupyter
(higgs_boson)>pip install numpy
(higgs_boson)>pip install pandas
(higgs_boson)>pip install matplotlib
```

Now, we can do our imports and image dispay setup.

In [1]:
import numpy  as np
import pandas as pd
from matplotlib import pyplot as plt

%matplotlib inline

Let's talk a little about the physics of the data we'll be looking at. You may know that Uranium can decay into lighter elements, as long as the combined mass-energy of the daughters (lighter elements' atoms, energy) is the same as the combined mass-energy - usually just the original mass - of the original Uranium.

The same happens for particles. The Higgs boson can decay into lighter particles and energy as long as its daughter decay particles and the energy is equal to the original mass-energy of the Higgs. One way that physics allows this to happen is for the Higgs to decay into four $\ell$eptons. (I use the script '$\ell$' to avoid confusion between '1' and 'l'.)

Here is a way physicists represent such a decay, which has the problems noted on the image.

<br/>
<div>
  <img src="./four_leptons_description_01.png"
       alt="Born (basically Feynman) diagrams for the two possible Higgs to four electrons scenario."
       width=100%>
</div>
<br/>

(In case you're curious, I'll tell you that the the $Z^{(*)}$ lines are for off-shell (virtual) Z-bosons. You can look up details if you'd like. Actually, to be technically correct, the Higgs boson should be off-shell (virtual), too, and we should see it labeled $H^{(*)}$. Just for fun, I'll put in this image from the [Wikipedia article on the Higgs boson](https://en.wikipedia.org/wiki/Higgs_boson#Discovery_of_candidate_boson_at_CERN) ([archived](https://web.archive.org/web/20230314192829/https://en.wikipedia.org/wiki/Higgs_boson)). It should show up even if changes are made to the wiki article and or the image, because I'm using an [archived image](https://web.archive.org/web/20230314233304/https://en.wikipedia.org/wiki/File:4-lepton_Higgs_decay.svg). This diagram doesn't show whether something is off-shell/virtual with the $^{(*)}$, because one learns that anything in the middle of such a diagram is virtual/off-shell.

<br/>
<div>
  <img src="https://web.archive.org/web/20230314233304/https://upload.wikimedia.org/wikipedia/commons/thumb/b/b2/4-lepton_Higgs_decay.svg/320px-4-lepton_Higgs_decay.svg.png"
       alt="Both sides of the diagram for Higgs to four leptons"
       width="480px">
</div>
<br/>

That represents to protons coming in and four leptons coming out.

In the first image, I said that we could simplify things while including more leptons than the electron. Let's use these symbols,

<br/>
<div>
  <img src="./four_leptons_description_02.png"
       alt="Symbols for electron, positron (positive version of the electron), muon, and antimuon (positive version of the muon)"
       width=300px>
</div>
<br/>

and give the three possibilites. We'll see these possibilities in the names of the `CSV` files.

2023-03-15

**Possibility 1: ... $H^{(*)} \rightarrow \{ 2 Z^{(*)} \} \rightarrow \{ 4e \}$**<br/>
**Higgs to 4 electrons (via two Z bosons and after previous interactions)**

<br/>
<div>
  <img src="./decay_four_leptons_four_electrons.png"
       alt="Off-mass-shell (virtual) Higgs to two off-mass-shell (virtual) Z bosons to four electrons"
       width=450px>
</div>
<br/>

**Possibility 2: ... $H^{(*)} \rightarrow \{ 2 Z^{(*)} \} \rightarrow \{ 4\mu$ \}**<br/>
**Higgs to four muons (via two Z bosons and after previous interactions)**

<br/>
<div>
  <img src="./decay_four_leptons_four_muons.png"
       alt="Off-mass-shell (virtual) Higgs to two off-mass-shell (virtual) Z bosons to four muons"
       width=450px>
</div>
<br/>

**Possibility 3: ... $H^{(*)} \rightarrow \{ 2 Z^{(*)} \} \rightarrow \{ 2e, 2\mu$ \}**<br/>
**Higgs to 4 electrons (via two Z bosons and after previous interactions)**

<br/>
<div>
  <img src="./decay_four_leptons_two_electrons_two_muons.png"
       alt="Off-mass-shell (virtual) Higgs to two off-mass-shell (virtual) Z bosons to four electrons"
       width=450px>
</div>
<br/>

Now, let's read in the CSV files for each of those three possibilities. We'll take a quick peek at the data, but not too much (there's a bunch). We'll look more carefully at a few of the columns and figure out the physics.

( _DWB note_, 20230315T175999-0600) Using `pd.read_csv` continues to get me `HTTPError: HTTP Error 504: Gateway Time-out` errors. I'm going to switch to using the `requests` module. I will keep my reference on some options from [this SO reference](https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests) ([archived](https://web.archive.org/web/20230316000241/https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests)).

After a git commit, I'll put the non-working code in this markdown cell and use the new code.

In [5]:
#4 electrons as daughter particles (two years' of data)
the_4e_lepton_participants_2011 = \
  pd.read_csv(
    'http://opendata.cern.ch/record/5200/files/4e_2011.csv')

In [ ]:
the_4e_lepton_participants_2012 = \
  pd.read_csv(
    'http://opendata.cern.ch/record/5200/files/4e_2012.csv')

In [ ]:
# 4 muons as daughter particles (two years' of data)
the_4mu_lepton_participants_2011 = \
  pd.read_csv(
    'http://opendata.cern.ch/record/5200/files/4mu_2011.csv')

In [ ]:
the_4mu_lepton_participants_2012 = \
  pd.read_csv(
    'http://opendata.cern.ch/record/5200/files/4mu_2012.csv')

In [ ]:
# 2 electrons and 2 muons as daughter particles (two years' of data)
the_2e2mu_lepton_participants_2011 = \
  pd.read_csv(
    'http://opendata.cern.ch/record/5200/files/2e2mu_2011.csv')

In [ ]:
the_2e2mu_lepton_participants_2012 = \
  pd.read_csv(
    'http://opendata.cern.ch/record/5200/files/2e2mu_2012.csv')

In [ ]:
##########################################
##  A peek at the data for 4e from 2011 ##
##+ Side scrolling will be necessary    ##
##########################################
the_4e_lepton_participants_2011.head()

In [4]:
##  DWB, 20230315T180500-0600
##+ Trying the strategy used in the original guide.
##+ orig_ref="https://opendata-education.github.io/en_Workshops/" + \
##+ "exercises/Hunting-the-Higgs-4leptons.html"
##+ orig_ref_archived=""
##+
##+ It didn't work, either (same `HTTPError: HTTP Error 504: Gateway Time-out`)

# Data for later use. 

csvs = [pd.read_csv('http://opendata.cern.ch/record/5200/files/4mu_2011.csv'), pd.read_csv('http://opendata.cern.ch/record/5200/files/4e_2011.csv'), pd.read_csv('http://opendata.cern.ch/record/5200/files/2e2mu_2011.csv')]
csvs += [pd.read_csv('http://opendata.cern.ch/record/5200/files/4mu_2012.csv'), pd.read_csv('http://opendata.cern.ch/record/5200/files/4e_2012.csv'), pd.read_csv('http://opendata.cern.ch/record/5200/files/2e2mu_2012.csv')]

fourlep = pd.concat(csvs)

HTTPError: HTTP Error 504: Gateway Time-out